In [1]:
from time import sleep
import json
from kafka import KafkaProducer
from kafka import KafkaConsumer

In [2]:
#channel
topic='app'

# producer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],value_serializer=lambda x: json.dumps(x).encode('utf-8'))

#consumer
consumer = KafkaConsumer(topic, bootstrap_servers=['localhost:9092'],auto_offset_reset='earliest',value_deserializer=lambda x:json.loads(x.decode('utf-8')))


In [3]:
for e in range(5):
    data = {'number' : e}
    producer.send(topic, value=data)
    sleep(1)

In [4]:
for message in consumer:
    message = message.value
    print('Message :{}'.format(message))

Message :{'number': 0}
Message :{'number': 1}
Message :{'number': 2}
Message :{'number': 3}
Message :{'number': 4}
